In [ ]:
'''
開始前，必須先去下載 GCP 為我們搜集的Log
並把名字改為 downloaded-logs.csv
上傳到colab

'''

In [ ]:
'''
讀取csv

python read csv
https://www.shanelynn.ie/python-pandas-read-csv-load-data-from-csv-files/
'''
import pandas as pd 
data = pd.read_csv("downloaded-logs.csv") 
data

In [ ]:
'''
抓出指定欄位 - jsonPayload.message

'''
data['jsonPayload.message']

In [ ]:
'''
將jsonPayload.message 的欄位內容 轉成csv

欄位若無值，則不填

'''
import json
json_data_list = []


df_message = pd.DataFrame(columns=['userId', 'timestamp', 'event_type','message_type','message_id', 'message_text'])

for row in  data['jsonPayload.message']:
  
  events = json.loads(row).get('events')[0]
  print(json.loads(row).get('events')[0])
  # json_data_list.append()
  # print(line_event.get('type'))
  # print(line_event.get('message'))
  # print(line_event.get('message').get('id'))
  # print(line_event.get('timestamp'))
  # print(line_event.get('source').get('userId'))
  if events.get('type') not in ['follow','unfollow','postback']:
    df_message = df_message.append(
        {
          'userId': events.get('source').get('userId'), 
          'timestamp': events.get('timestamp'), 
          'event_type': events.get('type'),
          'message_type': events.get('message').get('type'),
          'message_id': events.get('message').get('id'),
          'message_text': events.get('message').get('text')
        },
        ignore_index=True)
  elif events.get('type')  in ['postback']:
    df_message =df_message.append(
        {
          'userId': events.get('source').get('userId'), 
          'timestamp': events.get('timestamp'), 
          'event_type': events.get('type'),
          'message_text': events.get('postback').get('data')
        },
        ignore_index=True
    )
  else:
    df_message = df_message.append(
        {
          'userId': events.get('source').get('userId'), 
          'timestamp': events.get('timestamp'), 
          'event_type': events.get('type')
        },
        ignore_index=True)

df_message

In [ ]:
# 安裝line 套件
!pip install line-bot-sdk

In [ ]:
'''

設置Line Bot Api 的 Channel access token

'''

from linebot import (
    LineBotApi, WebhookHandler
)

line_bot_api = LineBotApi('CHANNEL_ACCESS_TOKEN')

In [ ]:
'''
取出用戶id，向Line要個資
整理成一份用戶個資名單
'''


user_info_df= pd.DataFrame(columns=['userId', 'display_name', 'picture_url','status_message'])

for user_id in df_message['userId'].unique():
  try:
    line_user_profile = line_bot_api.get_profile(user_id)
    
    user_info_df = user_info_df.append(
        {
          'display_name': line_user_profile.display_name, 
          'userId': line_user_profile.user_id, 
          'picture_url': line_user_profile.picture_url,
          'status_message': line_user_profile.status_message
        },
        ignore_index=True)
  except:
    pass

user_info_df

In [ ]:
'''

將用戶個資名單跟 jsonPayload的欄位內容 整理在一份csv

'''
merge_data = pd.merge(df_message, user_info_df)
merge_data

In [ ]:
'''
timestamp 轉換成人類可懂的時間
'''

import datetime
human_use_date=[]
for stamp in merge_data['timestamp']:
  
  timestamp = datetime.datetime.fromtimestamp(
      stamp/1000+8*3600
      )
  human_use_date.append(timestamp.strftime('%Y-%m-%d %H:%M:%S'))
  

merge_data['human_date']=human_use_date
merge_data

In [ ]:
'''
將資料存成csv
'''
merge_data.to_csv("final_data.csv")

In [ ]:
'''

進行可視化

外國人不考慮中文，中文會是亂碼

'''



In [ ]:
'''
看看誰發的話多
'''

merge_data['userId'].value_counts().sort_index().plot.bar()
# merge_data['timestamp'].plot.hist()

In [ ]:
'''
看看都發哪一類型的消息
'''
merge_data['message_type'].value_counts().sort_index().plot.bar()